In [1]:
import sys

import networkx as nx
from networkx.generators.community import LFR_benchmark_graph

import numpy as np
from scipy.integrate import odeint
from scipy.optimize import fsolve
from scipy.sparse.linalg import spsolve
import networkx as nx
from scipy.linalg import expm
import time


sys.path.append("/home/oriol/Documents/Trento/JacobianGeometry/Code/Dynamics_v2/")
import CommonFunctions as cf

In [2]:
#Creating ER network
N = 50
pp = 5./N #mean degree 5

G_er = nx.fast_gnp_random_graph(N, pp)
while nx.is_connected(G_er) == 0:
    print('The network should be in a single component. New trial!')
    G_er = nx.fast_gnp_random_graph(N, pp)

infoG = ['ER',pp]

print('Mean degree', np.mean(list(dict(G_er.degree()).values())))

Mean degree 5.36


In [3]:
def Simu(dynamics, G, infoG):
    """
    Inputs: dynamics (any from  dynamics_list = ['Mutualistic', 'Biochemical', 'Population', 'Regulatory',
                                                 'Epidemics','Synchronization', 'Neuronal', 'NoisyVM'])
            Network topology G
            infoG (list): Info about the network. infoG[0] = network_type; infoG[i>0] = parameters
    """
    start_time = time.time()
    
    num_nodes = G.number_of_nodes()
    num_time_points = 100 # small to decrease computational load
    times = np.linspace(0, 100, num = num_time_points) #time to find the steady state    
    str_tp = "np.logspace(-2, 2., num = num_time_points)" #times at which we look how the perturbation evolves
    times_perturbation = eval(str_tp)

    if dynamics in ['Mutualistic', 'Population', 'Regulatory','Synchronization', 'Neuronal']:
        perturbation_strength = 0.5
    elif dynamics in ['Biochemical', 'Epidemics', 'NoisyVM']:
        perturbation_strength = 0.05
    else:
        print('Dynamics not valid. Manual Exiting')
        exit()
    
    if nx.is_connected(G) == 0:
        raise ValueError('The network should be in a single component. Exit!')
    

    #Integration to get the steady state
    initial_state = np.random.random(len(G.nodes()))   
#    initial_state = np.ones(len(G.nodes()))

    SteadyState_ref = cf.Numerical_Integration(G, dynamics, initial_state, times = times, fixed_node = 1e+6, show = 1)
#        SteadyState_ref = SteadyState_ref[-1]/max(SteadyState_ref[-1])
    SteadyState_ref = SteadyState_ref[-1]

    d_t = cf.Jacobian(G, dynamics, SteadyState_ref, 
                      perturbation_strength, times_perturbation)

    mean_d = np.zeros(num_time_points)
    d_ij = np.zeros((num_time_points,num_nodes,num_nodes))
        
    c = 0
    for node1 in range(0, num_nodes):
      print(node1)
      for node2 in range(node1, num_nodes):
          if node1 != node2:
            Perturbed_SteadyState = cf.Numerical_Integration_perturbation(G,dynamics,SteadyState_ref,
                                                                          node1,node2, perturbation_strength,
                                                                          times = times_perturbation,
                                                                          fixed_node = 1e+6, show = 0)

#                delta1 = abs(Perturbed_SteadyState[:,node1] - SteadyState_ref[node1])
#                delta2 = abs(Perturbed_SteadyState[:,node2] - SteadyState_ref[node2])
#                d_tmp = abs(delta1 - delta2)
#                mean_d += d_tmp

            delta1 = Perturbed_SteadyState[:,node1] - SteadyState_ref[node1]
            delta2 = Perturbed_SteadyState[:,node2] - SteadyState_ref[node2]
            d_tmp = np.sqrt(delta1*delta1 + delta2*delta2)
            
#            if d_tmp != 0:
            c+=1
            mean_d += d_tmp

            if 0 in d_tmp:
                print('0 inside ', c)

            d_ij[:, node1,node2] = d_tmp
            
    d_ij = d_ij/c
    mean_d = mean_d/c
    
    cf.PrintFiles(mean_d, d_t, d_ij, times_perturbation, str_tp, dynamics, G, infoG)

    print("--- %s seconds ---" % (time.time() - start_time))

    return


In [4]:
dyn = 'Epidemics'

Simu(dyn,G_er,infoG)

/home/oriol/Documents/Trento/JacobianGeometry/Code/Dynamics/CommonFunctions.py:187: RuntimeWarning: invalid value encountered in true_divide
  J2 = J/J


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
['ER', 0.1]
--- 52.65021276473999 seconds ---


In [ ]:
dynamics_list = ['Mutualistic', 'Biochemical', 'Population', 'Regulatory', 
                 'Epidemics','Synchronization', 'Neuronal', 'NoisyVM']

for dyn in dynamics_list:
    print(dyn, 'dynamics')
    Simu(dyn,G_er,infoG)